Importing pandas ; Importing price prediction by book value growth csv ; Importing the price csv

In [1]:
import pandas as pd
df_prediction=pd.read_csv('../../data/output/price-prediction-by-book-value-growth.csv',sep=',')
df_price=pd.read_csv('../../data/input/price.csv',sep=',')

Dropping the prediction columns except the 3 Prediction column 

In [2]:
df_prediction=df_prediction[['Year','3 Prediction']]
df_prediction=df_prediction.set_index('Year')
print(df_prediction.head(len(df_prediction)))

      3 Prediction
Year              
2009           NaN
2010           NaN
2011           NaN
2012   1949.859262
2013   2238.857615
2014   2369.029813
2015   3225.604601
2016   3191.906402
2017   3746.859389
2018   5218.683060
2019   6211.703465
2020   7741.498318
2021   7587.731096
2022   8046.755950
2023   9262.144589
2024  10290.176731


Creating the price year index ; Calculating price means by month and year

In [3]:
df_price=df_price.drop(columns=['Open','High','Low','Volume','Adj Close'])

df_price['Year']=pd.to_datetime(df_price['Date']).dt.year
df_price=df_price.set_index('Year')
df_price['Month']=pd.to_datetime(df_price['Date']).dt.month
df_price=df_price.drop(columns=['Date'])

df_price=df_price.groupby(['Year','Month']).mean()
print(df_price.head(len(df_price)))

                   Close
Year Month              
2008 1        700.500000
     2        706.578947
     3        662.222222
     4        598.409091
     5        607.222222
...                  ...
2024 1       9576.136364
     2       9819.444444
     3      10056.944444
     4       9682.812500
     5       9655.000000

[197 rows x 1 columns]


Creating the progress dataframe

In [4]:
df_progress=df_price
df_progress['Progress']=df_progress['Close']/df_prediction['3 Prediction']

# make month index available as column
df_progress['Month']=[i[1] for i in df_progress.index]

Calculating lower bound and upper bound for progress dataframe cleansing

In [5]:
months=range(1,13)
lower_bound = []
upper_bound = []

for month in months:
    q1 = df_progress[df_progress['Month'] == month]['Progress'].quantile(0.25)
    q3 = df_progress[df_progress['Month'] == month]['Progress'].quantile(0.75)
    iqr = q3 - q1
    lower_bound.append(q1 - 1.5 * iqr)
    upper_bound.append(q3 + 1.5 * iqr)  

Cleaning the progress dataframe from outliers

In [6]:
df_cleaned_progress=pd.DataFrame()
for month in months:
    df_cleaned_progress=pd.concat([df_cleaned_progress,df_progress[(df_progress['Month'] == month) & (df_progress['Progress'] > lower_bound[month-1]) & (df_progress['Progress'] < upper_bound[month-1])]])

# dropping the month column
df_cleaned_progress=df_cleaned_progress.drop(columns=['Month','Close'])
df_cleaned_progress=df_cleaned_progress.sort_values(by=['Year','Month'])

print(df_cleaned_progress.head(len(df_cleaned_progress)))

            Progress
Year Month          
2012 1      0.825456
     2      0.783939
     3      0.802500
     4      0.814931
     5      0.785893
...              ...
2024 1      0.930610
     2      0.954254
     3      0.977334
     4      0.940976
     5      0.938273

[147 rows x 1 columns]


Exporting the progress dataframe csv

In [7]:
df_cleaned_progress.to_csv('../../data/output/monthly-progress-by-book-value/mean.csv',sep=';',index=True)